In [14]:
import pymongo
from pymongo import MongoClient
from datetime import datetime, timedelta
import random
import uuid
from bson.objectid import ObjectId

In [15]:
# MongoDB connection
client = MongoClient('mongodb://localhost:27017/')
db = client['p2p_lending_db']

In [16]:
# Define collections
users_collection = db['users']
loans_collection = db['loans']
transactions_collection = db['transactions']
lenders_collection = db['lenders']
borrowers_collection = db['borrowers']

In [17]:
# Create indexes for better query performance
users_collection.create_index('email', unique=True)
loans_collection.create_index('loan_id', unique=True)
loans_collection.create_index('user_id')
transactions_collection.create_index('transaction_id', unique=True)
transactions_collection.create_index('account_id')

'account_id_1'

In [18]:
# names
names = {
    'first_names': ['Sanjay', 'Chandana', 'Sakshi', 'Bhoomika', 'Prabal', 'Sneha', 'Nikhil', 'Akash', 'Nithik', 'ADITHYA', 
                    'Hardik', 'Suchitra', 'akshat', 'Archana', 'Deepak', 'Devi', 'ASHOK', 'Rohan', 'Imraz', 'PAVANKUMAR'],
    'last_names': ['Reddy', 'Ramanahali', 'Mayya', 'Hegde', 'Shetty', 'Shet', 'Gouda', 'chitragar', 'Joseph', 'ACHARYA', 
                   'prabhu', 'Desai', 'maurya', 'Halagannanavar', 'Rao', 'Prasad', 'BONI', 'Devang', 'Ziya', 'NAYAK']
}

In [19]:
def populate_sample_data():
    """Populate the database with sample data using Karnataka names"""
    # Clear existing data
    users_collection.delete_many({})
    loans_collection.delete_many({})
    transactions_collection.delete_many({})
    lenders_collection.delete_many({})
    borrowers_collection.delete_many({})
    
    # Add sample users in order
    sample_users = []
    for i in range(len(names['first_names'])):
        first_name = names['first_names'][i]
        last_name = names['last_names'][i]
        user = {
            'user_id': f'USR{i+1:04d}',
            'first_name': first_name,
            'last_name': last_name,
            'email': f'{first_name.lower()}{i}@email.com',
            'date_of_birth': datetime(random.randint(1970, 2000), random.randint(1, 12), random.randint(1, 28)),
            'registration_date': datetime.now() - timedelta(days=random.randint(30, 365)),
            'can_lend_after': datetime.now()  # By default, user can lend immediately
        }
        sample_users.append(user)
    
    users_collection.insert_many(sample_users)
    
    # Add sample borrowers and loans
    for i in range(10):
        borrower_user = sample_users[i]
        loan_amount = random.choice([50000, 100000, 150000, 200000, 250000, 300000])
        
        # Create borrower record
        borrower = {
            'user_id': borrower_user['user_id'],
            'total_borrowed': loan_amount,
            'active_loans': 1,
            'credit_score': random.randint(600, 800)
        }
        borrowers_collection.insert_one(borrower)
        
        # Create loan record
        loan_id = f'LN{i+1:04d}'
        loan = {
            'loan_id': loan_id,
            'user_id': borrower_user['user_id'],
            'loan_amount': loan_amount,
            'interest_rate': random.uniform(8.0, 15.0),
            'loan_term_months': random.choice([12, 24, 36]),
            'loan_start_date': datetime.now() - timedelta(days=random.randint(1, 365)),
            'loan_status': 'active',
            'lenders': []  # Will add lenders separately
        }
        
        # Select random lenders (ensuring they're not the borrower)
        available_lenders = [user for user in sample_users if user['user_id'] != borrower_user['user_id']]
        num_lenders = random.randint(1, 4)
        selected_lenders = random.sample(available_lenders, num_lenders)
        
        # Calculate amount per lender
        remaining_amount = loan_amount
        for idx, lender_user in enumerate(selected_lenders):
            if idx == len(selected_lenders) - 1:
                lender_amount = remaining_amount
            else:
                lender_amount = remaining_amount // (len(selected_lenders) - idx)
            
            remaining_amount -= lender_amount
            
            # Add lender to loan
            loan['lenders'].append({
                'user_id': lender_user['user_id'],
                'amount': lender_amount,
                'lend_date': loan['loan_start_date']
            })
            
            # Create/update lender record
            lender_record = lenders_collection.find_one({'user_id': lender_user['user_id']})
            if lender_record:
                lenders_collection.update_one(
                    {'user_id': lender_user['user_id']},
                    {'$inc': {'total_lent': lender_amount, 'active_loans': 1}}
                )
            else:
                lenders_collection.insert_one({
                    'user_id': lender_user['user_id'],
                    'total_lent': lender_amount,
                    'active_loans': 1,
                    'lending_score': random.randint(700, 900)
                })
            
            # Create transaction for lending
            transaction = {
                'transaction_id': f'TXN{str(uuid.uuid4())[:8]}',
                'account_id': lender_user['user_id'],
                'amount': lender_amount,
                'transaction_type': 'debit',
                'transaction_date': loan['loan_start_date'],
                'description': f'Lend to loan {loan_id}'
            }
            transactions_collection.insert_one(transaction)
        
        # Create transaction for borrowing
        borrow_transaction = {
            'transaction_id': f'TXN{str(uuid.uuid4())[:8]}',
            'account_id': borrower_user['user_id'],
            'amount': loan_amount,
            'transaction_type': 'credit',
            'transaction_date': loan['loan_start_date'],
            'description': f'Received loan {loan_id}'
        }
        transactions_collection.insert_one(borrow_transaction)
        
        # Update borrower's restriction on lending
        users_collection.update_one(
            {'user_id': borrower_user['user_id']},
            {'$set': {'can_lend_after': loan['loan_start_date'] + timedelta(days=730)}}  # 2 years restriction
        )
        
        loans_collection.insert_one(loan)
    
    print("Sample data populated successfully!")

In [20]:
def create_new_loan(user_id, loan_amount, interest_rate, loan_term_months, lender_info):
    """
    Create a new loan instance
    
    Args:
    - user_id: ID of the borrower
    - loan_amount: Total loan amount
    - interest_rate: Annual interest rate in percentage
    - loan_term_months: Loan term in months
    - lender_info: List of dictionaries containing lender user_id and amount
    
    Returns:
    - loan_id if successful, None otherwise
    """
    try:
        # Check if user exists and can take a loan
        user = users_collection.find_one({'user_id': user_id})
        if not user:
            print(f"User {user_id} not found")
            return None
        
        # Check if user is already a borrower with active loans
        borrower = borrowers_collection.find_one({'user_id': user_id})
        if borrower and borrower.get('active_loans', 0) > 0:
            print(f"User {user_id} already has active loans")
            return None
        
        # Generate new loan ID
        loan_count = loans_collection.count_documents({})
        loan_id = f'LN{loan_count+1:04d}'
        
        # Validate lenders and their amounts
        total_lender_amount = 0
        for lender in lender_info:
            lender_user = users_collection.find_one({'user_id': lender['user_id']})
            if not lender_user:
                print(f"Lender {lender['user_id']} not found")
                return None
            
            # Check if lender can lend (not restricted due to previous borrowing)
            if lender_user.get('can_lend_after', datetime.now()) > datetime.now():
                print(f"Lender {lender['user_id']} cannot lend yet")
                return None
            
            total_lender_amount += lender['amount']
        
        if total_lender_amount != loan_amount:
            print("Lender amounts don't match loan amount")
            return None
        
        # Create loan document
        loan = {
            'loan_id': loan_id,
            'user_id': user_id,
            'loan_amount': loan_amount,
            'interest_rate': interest_rate,
            'loan_term_months': loan_term_months,
            'loan_start_date': datetime.now(),
            'loan_status': 'active',
            'lenders': []
        }
        
        # Process each lender
        for lender in lender_info:
            # Add lender to loan
            loan['lenders'].append({
                'user_id': lender['user_id'],
                'amount': lender['amount'],
                'lend_date': datetime.now()
            })
            
            # Update or create lender record
            lender_record = lenders_collection.find_one({'user_id': lender['user_id']})
            if lender_record:
                lenders_collection.update_one(
                    {'user_id': lender['user_id']},
                    {'$inc': {'total_lent': lender['amount'], 'active_loans': 1}}
                )
            else:
                lenders_collection.insert_one({
                    'user_id': lender['user_id'],
                    'total_lent': lender['amount'],
                    'active_loans': 1,
                    'lending_score': 800  # Default score
                })
            
            # Create transaction for lending
            transaction = {
                'transaction_id': f'TXN{str(uuid.uuid4())[:8]}',
                'account_id': lender['user_id'],
                'amount': lender['amount'],
                'transaction_type': 'debit',
                'transaction_date': datetime.now(),
                'description': f'Lend to loan {loan_id}'
            }
            transactions_collection.insert_one(transaction)
        
        # Create or update borrower record
        if borrower:
            borrowers_collection.update_one(
                {'user_id': user_id},
                {'$inc': {'total_borrowed': loan_amount, 'active_loans': 1}}
            )
        else:
            borrowers_collection.insert_one({
                'user_id': user_id,
                'total_borrowed': loan_amount,
                'active_loans': 1,
                'credit_score': 750  # Default score
            })
        
        # Create transaction for borrowing
        borrow_transaction = {
            'transaction_id': f'TXN{str(uuid.uuid4())[:8]}',
            'account_id': user_id,
            'amount': loan_amount,
            'transaction_type': 'credit',
            'transaction_date': datetime.now(),
            'description': f'Received loan {loan_id}'
        }
        transactions_collection.insert_one(borrow_transaction)
        
        # Update borrower's restriction on lending
        users_collection.update_one(
            {'user_id': user_id},
            {'$set': {'can_lend_after': datetime.now() + timedelta(days=730)}}  # 2 years restriction
        )
        
        # Insert loan
        loans_collection.insert_one(loan)
        print(f"Loan {loan_id} created successfully")
        return loan_id
    
    except Exception as e:
        print(f"Error creating loan: {str(e)}")
        return None

In [21]:
def search_loan_by_id(loan_id):
    """
    Search for a loan account detail given LoanId
    
    Args:
    - loan_id: The ID of the loan to search for
    
    Returns:
    - Loan details if found, None otherwise
    """
    try:
        # Find loan
        loan = loans_collection.find_one({'loan_id': loan_id})
        if not loan:
            print(f"Loan {loan_id} not found")
            return None
        
        # Get borrower information
        borrower_user = users_collection.find_one({'user_id': loan['user_id']})
        borrower_record = borrowers_collection.find_one({'user_id': loan['user_id']})
        
        # Get lender information
        lender_details = []
        for lender in loan['lenders']:
            lender_user = users_collection.find_one({'user_id': lender['user_id']})
            lender_record = lenders_collection.find_one({'user_id': lender['user_id']})
            
            lender_details.append({
                'user_id': lender['user_id'],
                'name': f"{lender_user['first_name']} {lender_user['last_name']}",
                'amount': lender['amount'],
                'lend_date': lender['lend_date'],
                'lending_score': lender_record.get('lending_score', 'N/A')
            })
        
        # Get transactions
        transactions = list(transactions_collection.find({
            'description': {'$regex': f'{loan_id}'}
        }).sort('transaction_date', 1))
        
        # Compile loan details
        loan_details = {
            'loan_id': loan['loan_id'],
            'borrower': {
                'user_id': loan['user_id'],
                'name': f"{borrower_user['first_name']} {borrower_user['last_name']}",
                'email': borrower_user['email'],
                'credit_score': borrower_record.get('credit_score', 'N/A'),
                'total_borrowed': borrower_record.get('total_borrowed', 0)
            },
            'loan_amount': loan['loan_amount'],
            'interest_rate': loan['interest_rate'],
            'loan_term_months': loan['loan_term_months'],
            'loan_start_date': loan['loan_start_date'],
            'loan_status': loan['loan_status'],
            'lenders': lender_details,
            'transactions': [{
                'transaction_id': txn['transaction_id'],
                'account_id': txn['account_id'],
                'amount': txn['amount'],
                'type': txn['transaction_type'],
                'date': txn['transaction_date'],
                'description': txn['description']
            } for txn in transactions]
        }
        
        return loan_details
    
    except Exception as e:
        print(f"Error searching loan: {str(e)}")
        return None

In [22]:
def main():
    """Main function to demonstrate the P2P lending system"""
    # Populate sample data
    print("Populating sample data...")
    populate_sample_data()
    
    # Create a new loan
    print("\nCreating a new loan...")
    lender_info = [
        {'user_id': 'USR0015', 'amount': 100000},
        {'user_id': 'USR0016', 'amount': 50000}
    ]
    
    new_loan_id = create_new_loan(
        user_id='USR0017',
        loan_amount=150000,
        interest_rate=12.5,
        loan_term_months=24,
        lender_info=lender_info
    )
    
    if new_loan_id:
        print(f"New loan created with ID: {new_loan_id}")
        
        # Search for the newly created loan
        print(f"\nSearching for loan {new_loan_id}...")
        loan_details = search_loan_by_id(new_loan_id)
        
        if loan_details:
            print("\nLoan Details:")
            print(f"Loan ID: {loan_details['loan_id']}")
            print(f"Borrower: {loan_details['borrower']['name']} (ID: {loan_details['borrower']['user_id']})")
            print(f"Amount: ₹{loan_details['loan_amount']:,}")
            print(f"Interest Rate: {loan_details['interest_rate']}%")
            print(f"Term: {loan_details['loan_term_months']} months")
            print(f"Start Date: {loan_details['loan_start_date'].strftime('%Y-%m-%d')}")
            print(f"Status: {loan_details['loan_status']}")
            
            print("\nLenders:")
            for lender in loan_details['lenders']:
                print(f"- {lender['name']} (ID: {lender['user_id']}): ₹{lender['amount']:,}")
            
            print("\nTransactions:")
            for txn in loan_details['transactions']:
                print(f"- {txn['date'].strftime('%Y-%m-%d')}: {txn['type'].upper()} ₹{txn['amount']:,} ({txn['description']})")
    
    # Search for an existing loan
    print("\nSearching for an existing loan (LN0005)...")
    loan_details = search_loan_by_id('LN0005')
    if loan_details:
        print(f"Found loan: {loan_details['loan_id']}")
        print(f"Borrower: {loan_details['borrower']['name']}")
        print(f"Amount: ₹{loan_details['loan_amount']:,}")
        print(f"Status: {loan_details['loan_status']}")

if __name__ == "__main__":
    main()

Populating sample data...
Sample data populated successfully!

Creating a new loan...
Loan LN0011 created successfully
New loan created with ID: LN0011

Searching for loan LN0011...

Loan Details:
Loan ID: LN0011
Borrower: ASHOK BONI (ID: USR0017)
Amount: ₹150,000
Interest Rate: 12.5%
Term: 24 months
Start Date: 2025-04-19
Status: active

Lenders:
- Deepak Rao (ID: USR0015): ₹100,000
- Devi Prasad (ID: USR0016): ₹50,000

Transactions:
- 2025-04-19: DEBIT ₹100,000 (Lend to loan LN0011)
- 2025-04-19: DEBIT ₹50,000 (Lend to loan LN0011)
- 2025-04-19: CREDIT ₹150,000 (Received loan LN0011)

Searching for an existing loan (LN0005)...
Found loan: LN0005
Borrower: Prabal Shetty
Amount: ₹200,000
Status: active


In [ ]:
import logging

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()  # You can also add FileHandler to log to a file
    ]
)

def main():
    """Main function to demonstrate the P2P lending system"""
    logger = logging.getLogger(__name__)
    
    # Populate sample data
    logger.info("Populating sample data...")
    populate_sample_data()
    
    # Create a new loan
    logger.info("Creating a new loan...")
    lender_info = [
        {'user_id': 'USR0015', 'amount': 100000},
        {'user_id': 'USR0016', 'amount': 50000}
    ]
    
    new_loan_id = create_new_loan(
        user_id='USR0017',
        loan_amount=150000,
        interest_rate=12.5,
        loan_term_months=24,
        lender_info=lender_info
    )
    
    if new_loan_id:
        logger.info(f"New loan created with ID: {new_loan_id}")
        
        # Search for the newly created loan
        logger.info(f"Searching for loan {new_loan_id}...")
        loan_details = search_loan_by_id(new_loan_id)
        
        if loan_details:
            logger.info("Loan Details:")
            logger.info(f"Loan ID: {loan_details['loan_id']}")
            logger.info(f"Borrower: {loan_details['borrower']['name']} (ID: {loan_details['borrower']['user_id']})")
            logger.info(f"Amount: ₹{loan_details['loan_amount']:,}")
            logger.info(f"Interest Rate: {loan_details['interest_rate']}%")
            logger.info(f"Term: {loan_details['loan_term_months']} months")
            logger.info(f"Start Date: {loan_details['loan_start_date'].strftime('%Y-%m-%d')}")
            logger.info(f"Status: {loan_details['loan_status']}")
            
            logger.info("Lenders:")
            for lender in loan_details['lenders']:
                logger.info(f"- {lender['name']} (ID: {lender['user_id']}): ₹{lender['amount']:,}")
            
            logger.info("Transactions:")
            for txn in loan_details['transactions']:
                logger.info(f"- {txn['date'].strftime('%Y-%m-%d')}: {txn['type'].upper()} ₹{txn['amount']:,} ({txn['description']})")
    
    # Search for an existing loan
    logger.info("Searching for an existing loan (LN0005)...")
    loan_details = search_loan_by_id('LN0005')
    if loan_details:
        logger.info(f"Found loan: {loan_details['loan_id']}")
        logger.info(f"Borrower: {loan_details['borrower']['name']}")
        logger.info(f"Amount: ₹{loan_details['loan_amount']:,}")
        logger.info(f"Status: {loan_details['loan_status']}")

if __name__ == "__main__":
    main()

2025-04-19 18:18:45,036 - INFO - Populating sample data...
2025-04-19 18:18:45,067 - INFO - Creating a new loan...
2025-04-19 18:18:45,074 - INFO - New loan created with ID: LN0011
2025-04-19 18:18:45,075 - INFO - Searching for loan LN0011...
2025-04-19 18:18:45,077 - INFO - Loan Details:
2025-04-19 18:18:45,077 - INFO - Loan ID: LN0011
2025-04-19 18:18:45,077 - INFO - Borrower: ASHOK BONI (ID: USR0017)
2025-04-19 18:18:45,078 - INFO - Amount: ₹150,000
2025-04-19 18:18:45,078 - INFO - Interest Rate: 12.5%
2025-04-19 18:18:45,078 - INFO - Term: 24 months
2025-04-19 18:18:45,079 - INFO - Start Date: 2025-04-19
2025-04-19 18:18:45,079 - INFO - Status: active
2025-04-19 18:18:45,080 - INFO - Lenders:
2025-04-19 18:18:45,080 - INFO - - Deepak Rao (ID: USR0015): ₹100,000
2025-04-19 18:18:45,081 - INFO - - Devi Prasad (ID: USR0016): ₹50,000
2025-04-19 18:18:45,081 - INFO - Transactions:
2025-04-19 18:18:45,081 - INFO - - 2025-04-19: DEBIT ₹100,000 (Lend to loan LN0011)
2025-04-19 18:18:45,082

Sample data populated successfully!
Loan LN0011 created successfully
